# Clase Práctica: La Importancia de la Limpieza de Texto en NLP

En este notebook aprenderemos **por qué** limpiar los datos es el paso más importante en cualquier proyecto de NLP. Veremos cómo el texto "sucio" infla nuestro vocabulario inútilmente y cómo técnicas simples pueden mejorar drásticamente la calidad de nuestros datos.

## Objetivos
1.  Identificar ruido en textos (HTML, emails, signos).
2.  Normalizar texto (acentos, mayúsculas).
3.  Comparar el tamaño del vocabulario antes y después de la limpieza.

---

## 1. Setup e Instalación
Necesitaremos `spaCy` y su modelo en español.

In [ ]:
!pip install spacy sentence-transformers scikit-learn -q
!python -m spacy download es_core_news_sm

In [ ]:
import spacy
import re
import unicodedata
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Cargar modelo de SpaCy en español
nlp = spacy.load("es_core_news_sm")

## 2. El Problema: Texto "Sucio" del Mundo Real

In [ ]:
dirty_texts = [
    "¡HOLA! Estoy muy freliz con mi compra en www.tienda.com :)",
    "Hola, estoy feliz??? con la compra... Contacto: soporte@email.com",
    "Féliz de comprar aqui. #Recomendado 100%!!",
    "No me GUSTA nada, quiero devolución"
]

print("Ejemplos sucios:", dirty_texts)

### Análisis Inicial
Si intentamos contar palabras ahora, veremos el caos.

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(dirty_texts)

print(f"Tamaño del Vocabulario Sucio: {len(vectorizer.get_feature_names_out())}")
print("Algunas 'features':", vectorizer.get_feature_names_out())

**Observación:** Es probable que veas palabras separadas inútilmente o conectadas a signos, e incluso inconsistencias de mayúsculas.

## 3. La Caja de Herramientas de Limpieza

In [ ]:
def clean_text(text):
    # 1. Minúsculas
    text = text.lower()
    
    # 2. Eliminar URLs y Emails con Regex
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    
    # 3. Eliminar caracteres especiales (manteniendo letras y espacios básicos)
    # Forma simple: conservar solo alfanuméricos
    # text = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ ]', '', text)
    
    # 4. Normalización de Unicode (Quitar tildes si se desea, opcional)
    # text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # 5. Procesamiento con SpaCy (Lematización y Stopwords)
    doc = nlp(text)
    clean_tokens = []
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.like_num:
            # Usamos el lema (raíz) de la palabra
            clean_tokens.append(token.lemma_)
            
    return " ".join(clean_tokens)

# Probemos con uno
ejemplo = "¡Los niños están corriendo rápido!"
print(f"Original: {ejemplo}")
print(f"Limpio:   {clean_text(ejemplo)}")

## 4. Midiendo el Impacto

In [ ]:
# Aplicar limpieza a todo el corpus
cleaned_texts = [clean_text(t) for t in dirty_texts]

print("Textos Limpios:")
for t in cleaned_texts:
    print(f"- {t}")

In [ ]:
vectorizer_clean = CountVectorizer()
X_clean = vectorizer_clean.fit_transform(cleaned_texts)

print(f"Tamaño del Vocabulario Limpio: {len(vectorizer_clean.get_feature_names_out())}")
print("Features limpias:", vectorizer_clean.get_feature_names_out())

### Conclusión Visual
Hemos reducido el ruido y unificado conceptos (ej. "feliz" y "féliz" o conjugaciones verbales si aplicamos lematización correctamente).